In [25]:
# !pip install -r "E:\Tasks\Webinar\agentdemo\requirements.txt"

In [26]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dotenv import load_dotenv
import asyncio
from azure.identity import DefaultAzureCredential, get_bearer_token_provider


from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool

import os

load_dotenv()

True

In [27]:
token_provider = get_bearer_token_provider(  
    DefaultAzureCredential()
)  
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment=os.getenv("AZURE_API_DEPLOYMENT"),
    api_version=os.getenv("AZURE_API_VERSION"),
    model = "gpt-4o",
    azure_endpoint=os.getenv("AZURE_API_ENDPOINT"),
    # azure_ad_token_provider=token_provider,  # Optional if you choose key-based authentication.
    api_key=os.getenv("AZURE_API_KEY"), # For key-based authentication.
)

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.getenv("AZURE_PROJ_CONN"),
)

In [28]:
async def save_blog_agent(blog_content: str) -> str:

    print("This is Code Interpreter for Azure AI Agent Service .......")


    code_interpreter = CodeInterpreterTool()
        
    agent = project_client.agents.create_agent(
            model="gpt-4o-mini",
            name="my-agent",
            instructions="You are helpful agent",
            tools=code_interpreter.definitions,
            # tool_resources=code_interpreter.resources,
    )

    thread = project_client.agents.create_thread()

    message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content="""
        
                    You are my Python programming assistant. Generate code,save """+ blog_content +
                    
                """    
                    and execute it according to the following requirements

                    1. Save blog content to blog-{YYMMDDHHMMSS}.md

                    2. give me the download this file link
                """,
    )
    # create and execute a run
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
            # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

        # # delete the original file from the agent to free up space (note: this does not delete your version of the file)
        # project_client.agents.delete_file(file.id)
        # print("Deleted file")

        # print the messages from the agent
    messages = project_client.agents.get_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

        # get the most recent message from the assistant
    last_msg = messages.get_last_text_message_by_sender("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

        # print(f"File: {messages.file_path_annotations}")


    for file_path_annotation in messages.file_path_annotations:

        file_name = os.path.basename(file_path_annotation.text)

        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./blog")
        

    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")


        # project_client.close()


    return "Saved"

In [29]:
save_blog_content_agent = AssistantAgent(
    name="save_blog_content_agent",
    model_client=az_model_client,
    tools=[save_blog_agent],
    system_message="""
        Save blog content. Respond with 'Saved' to when your blog are saved.
    """
)

In [30]:
write_agent = AssistantAgent(
    name="write_agent",
    model_client=az_model_client,
    system_message="""
        You are a blog writer, please help me write a blog based on bing search content."
    """
)

In [31]:
text_termination = TextMentionTermination("Saved")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(10)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination


reflection_team = RoundRobinGroupChat([write_agent,save_blog_content_agent], termination_condition=termination)

In [32]:
async def run_task():
    await Console(
        reflection_team.run_stream(task="""
            I need career guidance.  
            I'm a third year student studing in Electronics Department. 
            Could you please help me on how to prepare for my placements?
        """)
    )

# Run the async function
await run_task()

---------- user ----------

            I need career guidance.  
            I'm a third year student studing in Electronics Department. 
            Could you please help me on how to prepare for my placements?
        
---------- write_agent ----------
**How to Prepare for Placements as an Electronics Engineering Student: A Guide for Third-Year Students**

As a third-year student in the field of Electronics Engineering, you're at a critical juncture in your academic journey. The steps you take now will shape your readiness for campus placements in the coming year. Preparing for placements can seem overwhelming, but with the right strategy and consistent effort, you can position yourself to stand out to recruiters. Here's a comprehensive guide to help you get started and excel in your placement process.

---

### **1. Understand What Recruiters Look For**
Recruiters hiring electronics engineers typically evaluate three main criteria:
   - **Technical Knowledge**: Your understanding o

e:\Personal\Kishore\Career\AzureDeveloperCommunity\CommunityEvents\Feb-15-MSUS\AIAgentCode\.venv\Lib\site-packages\autogen_agentchat\agents\_assistant_agent.py:307: UserWarning: Resolved model mismatch: gpt-4o-2024-08-06 != gpt-4o-2024-11-20. Model mapping may be incorrect.
  result = await self._model_client.create(


This is Code Interpreter for Azure AI Agent Service .......
Run finished with status: RunStatus.COMPLETED
Messages: <azure.ai.projects.models._patch.ThreadMessages object at 0x000002B7C8FDE510>
Last Message: The blog content has been saved successfully. You can download it using the link below:

[Download the blog file](sandbox:/mnt/data/blog-250215075153.md)
Deleted agent
---------- save_blog_content_agent ----------
[FunctionCall(id='call_6c093C7Gj8Rxyk2qjRSVf9jz', arguments='{"blog_content":"**How to Prepare for Placements as an Electronics Engineering Student: A Guide for Third-Year Students**\\n\\nAs a third-year student in the field of Electronics Engineering, you\'re at a critical juncture in your academic journey. The steps you take now will shape your readiness for campus placements in the coming year. Preparing for placements can seem overwhelming, but with the right strategy and consistent effort, you can position yourself to stand out to recruiters. Here\'s a comprehensive 